In [40]:
import json
import time
from datetime import datetime, timedelta

import pandas as pd
import requests

In [41]:
class GetApiError(Exception):
    def __init__(self, msg):
        self.msg = msg

    def __str__(self):
        return self.msg

In [42]:
now = datetime.now()

def get_servicekey() -> str:
    with open("secrets.json") as f:
        secrets = json.loads(f.read())
    return secrets.get('decoding_key')


def get_current_date() -> str:
    return now.strftime('%Y%m%d') + '00'


def get_tmFc() -> str: # 06시, 18시 가능
    if 0 <= now.hour < 6: # 전날 18시를 가져와야 함
        yesterday = (now + timedelta(days=-1))
        return yesterday.strftime('%Y%m%d1800')

    elif 6 <= now.hour < 18: # 당일 06시를 가져와야 함
        return now.strftime('%Y%m%d0600')

    elif 18 <= now.hour <= 24: # 당일 18시를 가져와야 함
        return now.strftime('%Y%m%d1800')

In [43]:
def get_temp_api(regId: str) -> list[dict]:
    temp_url = 'http://apis.data.go.kr/1360000/MidFcstInfoService/getMidTa'
    params = {'ServiceKey': get_servicekey(),
              'pageNo': '1',
              'numOfRows': '100',
              'dataType': 'JSON',
              'regId' : regId,
              'tmFc': get_tmFc()}
    while True:
        try:
            response = requests.get(url=temp_url, params=params, timeout=5).json()
            break
        except:
            print("get_temp_api error")
            time.sleep(3)

    if response['response']['header']['resultCode'] != '00':
        ErrorMessage = response['response']['header']['resultMsg']
        raise GetApiError(ErrorMessage)

    return response['response']['body']['items']['item']

In [44]:
def get_yooksang_api(regId: str) -> list[dict]:
    yooksang_url = 'http://apis.data.go.kr/1360000/MidFcstInfoService/getMidLandFcst'
    params = {'ServiceKey': get_servicekey(),
              'pageNo': '1',
              'numOfRows': '100',
              'dataType': 'JSON',
              'regId' : regId,
              'tmFc': get_tmFc()}

    while True:
        try:
            response = requests.get(url=yooksang_url, params=params, timeout=5).json()
            break
        except:
            print("get_yooksang_api error")
            time.sleep(3)

    if response['response']['header']['resultCode'] != '00':
        ErrorMessage = response['response']['header']['resultMsg']
        raise GetApiError(ErrorMessage)

    return response['response']['body']['items']['item']

In [45]:
def get_tour_api(city_area_id: str) -> list[dict]:
    url = 'http://apis.data.go.kr/1360000/TourStnInfoService1/getCityTourClmIdx1'
    params = {'ServiceKey': get_servicekey(),
             'pageNo': '1',
             'numOfRows': '100',
             'dataType': 'JSON',
            'CURRENT_DATE': get_current_date(),
            'DAY': '14',
            'CITY_AREA_ID': city_area_id}

    while True:
        try:
            response = requests.get(url=url, params=params, timeout=5).json()
            break
        except:
            print("get_tour_api error")
            time.sleep(3)


    if response['response']['header']['resultCode'] != '00':
        ErrorMessage = response['response']['header']['resultMsg']
        raise GetApiError(ErrorMessage)

    return response['response']['body']['items']['item']

In [46]:
# code_temp에 순천, 순천시가 따로 있다..
# 고성은 강원고성, 경남고성이 따로있다..
# 강원고성 : 4282..
# 경남고성 : 4882..

def make_merged_csv():
    df_temp = pd.read_csv('code_temp.csv')
    df_yooksang = pd.read_csv('code_yooksang.csv')
    df_info = pd.read_csv('code_informations.csv')
    df_info['시군구 아이디'] = df_info['시군구 아이디'].astype(str)

    exception_list = ['동구', '서구', '남구', '북구', '중구'] # 시군구 이름 전처리 (동구, 서구, 남구, 북구, 중구 제외하고 군/구/시 를 떼준다)
    df_info['새로운시군구이름'] = df_info['시군구이름'].apply(lambda x: x if x in exception_list else x[:-1])

    # df_merged에 df_info와 df_temp를 '도시'를 기준으로 outer join. (info에만 있는 정보가 있을 것이고, temp에만 있는 정보가 있을 것)
    df_merged = df_info.merge(df_temp, left_on='새로운시군구이름', right_on='도시', how='outer')

    # info만 있는 도시라면, 기온코드는 NaN일 것이므로, 'FFFFFFFF'로 대체해준다.
    df_merged['기온코드'].fillna('FFFFFFFF', inplace=True)

    # 기온코드가 11B로 시작한다면, 육상코드는 뒤에 00000이 온다.
    # 만약 그렇지 않다면, 11C10000 와 같이, 뒤에 0000이 온다.
    # 만약 육상코드가 존재하지 않는 값이라면, 'FFFFFFFF'로 설정해준다.
    df_merged['육상코드'] = df_merged['기온코드']\
            .apply(lambda x: x[:3]+'00000' if x and x.startswith('11B') else x[:4]+'0000')\
            .apply(lambda x: 'F'*8 if x not in df_yooksang['육상코드'].values else x)

    # 기온코드가 '21F'로 시작한다면, 육상코드는 '11F10000' 임. (전라도 중에, 기온코드가 들쑥날쑥한 것이 있다.)
    df_merged['육상코드'] = df_merged['육상코드'].apply(lambda x: '11F10000' if x.startswith('21F') else x)

    # 전체도시이름과 도단위이름이 비어있다면, (즉, info에 존재하지 않았던 도시라면,) 도시 이름으로 설정해준다.
    df_merged['전체도시이름'].fillna(df_merged['도시'], inplace=True)
    df_merged['도단위이름'].fillna(df_merged['도시'], inplace=True)

    # 필요없는 column을 제거한다.
    df_merged.drop(columns=['새로운시군구이름', '도시'], inplace=True)

    # column의 배치를 정렬한다.
    df_merged = df_merged[['전체도시이름', '도단위이름', '시군구이름', '기온코드', '육상코드', '시군구 아이디']]

    # column을 rename.
    df_merged.rename(columns={'시군구 아이디' : '투어코드'}, inplace=True)

    # FFFFFFFF 값과, NaN값들을, 빈 문자열로 대체한다.
    df_merged = df_merged.replace('FFFFFFFF', '').fillna('')

    # df_merged의 dataframe에 빈 값들을 '도시'와, '광역시/도'의 정보들로 치환
    new_df = pd.merge(left=df_merged, right=df_temp, left_on='도단위이름', right_on='도시', how='left')
    new_df['기온코드_x'].update(new_df['기온코드_y'])

    new_df = pd.merge(left=new_df, right=df_yooksang, left_on='도단위이름', right_on='광역시/도', how='left')
    new_df['육상코드_x'].update(new_df['육상코드_y'])

    new_df.drop(columns=['기온코드_y', '도시', '육상코드_y', '광역시/도'], inplace=True)
    new_df.rename(columns={'기온코드_x': '기온코드', '육상코드_x': '육상코드'}, inplace=True)
    new_df.dropna(inplace=True)

    new_df.to_csv('code_merged.csv', encoding='utf-8', index=False)

    df = pd.read_csv('code_merged.csv', dtype=str)
    df.dropna(inplace=True)
    df.to_csv('code_merged.csv')

In [47]:
def get_tour_dataframe(city_area_id: str) -> pd.DataFrame:

    tour_data = []
    tour_api_data = get_tour_api(city_area_id=city_area_id)

    for items in tour_api_data:
        day = items['tm'].split()[0]
        kmaTci = items['kmaTci']
        TCI_GRADE = items['TCI_GRADE']
        tour_data.append({'날짜': day, '투어코드': city_area_id, '관광지수': kmaTci, '관광지표': TCI_GRADE})

    return pd.DataFrame(tour_data)

In [48]:
def get_temp_dataframe(regId: str) -> pd.DataFrame:

    temp_data = []
    temp_api_data = get_temp_api(regId=regId)[0]

    # 관광지수는 9일까지만 제공하므로, 통일해준다. (max: 10이지만, 9까지만 돌림)
    for i in range(3, 10):
        day = now + timedelta(days=i-1)
        taMin = temp_api_data[f'taMin{i}']
        taMax = temp_api_data[f'taMax{i}']
        temp_data.append({'날짜': day.strftime('%Y-%m-%d'), '기온코드': regId, '최소기온': str(taMin), '최대기온': str(taMax)})

    return pd.DataFrame(temp_data)

In [49]:
def get_yooksang_dataframe(regId: str) -> pd.DataFrame:

    yooksang_data = []
    yooksang_api_data = get_yooksang_api(regId)[0]

    # am, pm은 7일까지만 제공, 데이터의 통일성을 위해, am, pm을 설정해준다.
    # 관광지수는 9일까지만 제공하므로, 통일해준다. (max: 10이지만, 9까지만 돌림)
    for i in range(3, 10):
        day = now + timedelta(days=i-1)
        if i <= 7:
            rain_am = yooksang_api_data[f'rnSt{i}Am']
            rain_pm = yooksang_api_data[f'rnSt{i}Pm']
            forecast_am = yooksang_api_data[f'wf{i}Am']
            forecast_pm = yooksang_api_data[f'wf{i}Pm']
        else:
            rain_am = yooksang_api_data[f'rnSt{i}']
            rain_pm = yooksang_api_data[f'rnSt{i}']
            forecast_am = yooksang_api_data[f'wf{i}']
            forecast_pm = yooksang_api_data[f'wf{i}']
        yooksang_data.append({'날짜': day.strftime('%Y-%m-%d'), '육상코드': regId,
                              '오전비확률': f'{rain_am}%', '오후비확률': f'{rain_pm}%',
                              '오전날씨': forecast_am, '오후날씨': forecast_pm})

    return pd.DataFrame(yooksang_data)

In [50]:
def get_total_dataframe():
    df_total = pd.DataFrame()
    df_merged = pd.read_csv('code_merged.csv', dtype={'투어코드': str})
    counter = 1

    for row in df_merged.iterrows(): # row : [idx, {column1 : value1, .. }]
        print(counter)
        counter += 1
        code_tour = row[1]['투어코드']
        code_temp = row[1]['기온코드']
        code_yooksang = row[1]['육상코드']
        df = pd.DataFrame(row[1]).T

        if code_tour != '':
            df_tour = get_tour_dataframe(code_tour)
            df = pd.merge(left=df, right=df_tour, on='투어코드', how='left')

        if code_temp != '':
            df_temp = get_temp_dataframe(code_temp)
            df = pd.merge(left=df, right=df_temp, on=['기온코드', '날짜'], how='left')

        if code_yooksang != '':
            df_yooksang = get_yooksang_dataframe(code_yooksang)
            df = pd.merge(left=df, right=df_yooksang, on=['육상코드', '날짜'], how='left')

        df_total = pd.concat([df_total, df])

    return df_total

In [ ]:
# === main start ===

In [51]:
%%time

if __name__ == "__main__":
    columns = ['날짜', '전체도시이름', '도단위이름', '시군구이름', '최소기온', '최대기온', '오전날씨', '오전비확률', '오후날씨', '오후비확률', '관광지수', '관광지표']
    df_total = get_total_dataframe()
    df_total = df_total[columns]
    df_total.dropna(inplace=True)
    df_total.set_index('날짜', inplace=True)
    df_total.sort_values('날짜', inplace=True)
    df_total.to_csv('city_with_weather.csv', encoding='utf-8')

1
2
3
4
5
6
7
8
get_temp_api error
9
10
11
12
13
14
get_yooksang_api error
15
16
17
18
19
get_temp_api error
get_temp_api error
20
21
22
23
get_yooksang_api error
24
25
get_yooksang_api error
get_yooksang_api error
26
27
28
get_temp_api error
get_yooksang_api error
29
get_tour_api error
30
31
32
get_tour_api error
33
get_tour_api error
34
35
36
37
38
39
40
get_yooksang_api error
41
42
43
44
45
46
47
48
get_yooksang_api error
49
get_temp_api error
50
51
52
53
get_temp_api error
get_yooksang_api error
54
55
get_temp_api error
56
57
get_yooksang_api error
get_yooksang_api error
58
59
60
get_yooksang_api error
61
62
63
64
65
66
67
68
69
70
71
72
73
get_tour_api error
74
75
76
get_tour_api error
77
get_tour_api error
get_tour_api error
78
79
80
81
82
83
84
get_tour_api error
85
get_temp_api error
86
get_yooksang_api error
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
get_yooksang_api error
133
134
135
1

KeyboardInterrupt: 